## 타깃: 마비노기_모바일

## 데이터 로드
- overall_ad_scores.csv
  - (https://colab.research.google.com/drive/1tV1F06Bz2_Cq4cIKBaSA5T01Zzxpmezt?usp=sharing)
  
  - 해당 주소의 코랩(모델_제작)에서 가져온 임의 결과물(행정동별 광고효율점수)
- 서울시_행정동_중심점_2017.csv - 코드, 시명,  시군구명, 읍면동명, X,Y 값을 가지고 있음(매핑이 가능할 경우 해당 좌표를 통해 맵 시각화가 가능)

- 현재 overall_ad_scores.csv 파일은 이전 모델에서 행정동코드를 변환하였음


In [ ]:
df = pd.read_csv("overall_ad_scores.csv")
df

,순위,행정동코드,행정동명,광고_효율_점수 (0-10점만점)
0,1,11440660,서교동,10.000000
1,2,11680640,역삼1동,8.989437
2,3,11560540,여의동,7.684371
3,4,11545510,가산동,6.476337
4,5,11410585,신촌동,5.954539
...,...,...,...,...
419,420,11470610,신월6동,0.080096
420,421,11305606,번3동,0.070593
421,422,11740690,둔촌1동,0.063255
422,423,11110680,창신2동,0.051708


In [ ]:
df_2 = pd.read_csv('서울시_행정동_중심점_2017.csv',encoding='euc-kr')
df_2

,코드,시도명,시군구명,읍면동명,X,Y
0,11110515,서울특별시,종로구,청운효자동,126.970417,37.584658
1,11110530,서울특별시,종로구,사직동,126.970144,37.574108
2,11110540,서울특별시,종로구,삼청동,126.981114,37.588013
3,11110550,서울특별시,종로구,부암동,126.962557,37.596699
4,11110560,서울특별시,종로구,평창동,126.969274,37.613965
...,...,...,...,...,...,...
419,11740650,서울특별시,강동구,성내제2동,127.127737,37.534365
420,11740660,서울특별시,강동구,성내제3동,127.133797,37.528404
421,11740685,서울특별시,강동구,길동,127.145907,37.539725
422,11740690,서울특별시,강동구,둔촌제1동,127.140435,37.523145


In [ ]:
# df와 df_2의 행정동 코드 컬럼 추출
df_codes = df['행정동코드']
df_2_codes = df_2['코드']

# 두 컬럼의 중복 없는 고유한 코드 확인
unique_df_codes = set(df_codes)
unique_df_2_codes = set(df_2_codes)

# 두 데이터프레임에 모두 존재하는 공통 코드 수 계산
common_codes = unique_df_codes.intersection(unique_df_2_codes)
num_common_codes = len(common_codes)

# 각 데이터프레임의 전체 고유 코드 수
num_df_codes = len(unique_df_codes)
num_df_2_codes = len(unique_df_2_codes)

# 일치율 계산 (두 데이터프레임 중 작은 쪽을 기준으로 일치율 계산)
match_rate = (num_common_codes / min(num_df_codes, num_df_2_codes)) * 100 if min(num_df_codes, num_df_2_codes) > 0 else 0

print(f"df의 고유 행정동 코드 수: {num_df_codes}")
print(f"df_2의 고유 행정동 코드 수: {num_df_2_codes}")
print(f"두 데이터프레임에 모두 존재하는 공통 코드 수: {num_common_codes}")
print(f"행정동 코드 매칭 일치율 (작은 데이터프레임 기준): {match_rate:.2f}%")

df의 고유 행정동 코드 수: 424
df_2의 고유 행정동 코드 수: 424
두 데이터프레임에 모두 존재하는 공통 코드 수: 424
행정동 코드 매칭 일치율 (작은 데이터프레임 기준): 100.00%


- 두 데이터의 행정동 코드가 100% 일치함 -> 매핑 진행 가능

In [ ]:
# df와 df_2 데이터프레임을 행정동 코드를 기준으로 병합
# df의 '행정동코드'와 df_2의 '코드' 컬럼을 사용합니다.
# df_2에서 필요한 '코드', 'X', 'Y' 컬럼만 선택하여 병합합니다.
df_merged = pd.merge(df, df_2[['코드', 'X', 'Y']], left_on='행정동코드', right_on='코드', how='left')

# 병합 후 더 이상 필요하지 않은 '코드' 컬럼은 제거합니다.
df_merged.drop(columns=['코드'], inplace=True)

# 병합 결과 확인
display(df_merged.head())

,순위,행정동코드,행정동명,광고_효율_점수 (0-10점만점),X,Y
0,1,11440660,서교동,10.000000,126.920948,37.554772
1,2,11680640,역삼1동,8.989437,127.035491,37.500889
2,3,11560540,여의동,7.684371,126.927571,37.527489
3,4,11545510,가산동,6.476337,126.884341,37.476835
4,5,11410585,신촌동,5.954539,126.941063,37.563636


In [ ]:
df_merged.to_csv('광고효율_좌표.csv',index=False)

## 데이터별 열 이름

- 인구:
  - ['연월', '행정동코드', '행정동명', '총생활인구수', '남자15세부터19세생활인구수', '남자20세부터24세생활인구수',
       '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
       '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '남자50세부터54세생활인구수',
       '남자55세부터59세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수',
       '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
       '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수',
       '여자55세부터59세생활인구수']

- 소득소비:
  - ['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '월_평균_소득_금액', '소득_구간_코드', '지출_총금액',
       '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액',
       '교육_지출_총금액', '여가_문화_지출_총금액', '음식_지출_총금액']

- 집객시설:
  - ['행정동_코드', '행정동_코드_명', '집객시설_수', '은행_수', '종합병원_수', '일반_병원_수', '약국_수',
       '유치원_수', '초등학교_수', '중학교_수', '고등학교_수', '대학교_수', '백화점_수', '슈퍼마켓_수',
       '극장_수', '숙박_시설_수', '지하철_역_수', '버스_정거장_수']

- 점포수:
  - ['행정동_코드', '행정동_코드_명', '음식점', '주점', '카페', '학원', '병원', '자동차', '미용', '의류']

#### 타깃:
    ad_target_info = {
        '인구': [
            '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수',
            '여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수'
        ],
        '소득소비': [
            '여가_문화_지출_총금액',
            '의류_신발_지출_총금액',
            '음식_지출_총금액'
        ],
        '집객시설': [
            '대학교_수',
            '지하철_역_수',
            '버스_정거장_수'
        ],
        '점포': [
            '카페'
        ]
    }

#### 맵 거품 색상
  🔹 단색 계열 (Single Hue Sequential)

  값이 낮으면 연한 색, 높으면 진한 색 → 단일 지표(점수, 인구 등)에 직관적

  linear.Reds_09 → 흰색 → 분홍 → 진한 빨강
  → 경고/위험, 열 지도, 점수 강조용

  linear.Blues_09 → 흰색 → 하늘색 → 진한 파랑
  → 수량, 차분한 데이터, 신뢰감 표현

  linear.Greens_09 → 흰색 → 연두 → 진한 초록
  → 환경, 긍정, 성장 지표 강조

  linear.Oranges_09 → 흰색 → 연주황 → 진한 주황
  → 에너지, 활동량, 활발한 데이터 표현

  linear.Purples_09 → 흰색 → 연보라 → 진보라
  → 창의성, 독창성, 인구 특성 표현

  linear.Greys_09 → 흰색 → 회색 → 검정
  → 중립적, 보조 색상, 음영/밀도 표현

  🔹 다색 연속형 (Multi-hue Sequential)

  밝은 색에서 어두운 색으로 연속 변환 → 분포/강도 비교에 적합

  linear.YlGn_09 → 노랑 → 연초록 → 진한 초록
  → 농업, 친환경 데이터, 긍정적 지표

  linear.YlOrRd_09 → 노랑 → 주황 → 빨강
  → 단계적 경고, 위험 등급, 집중도 표현

  linear.YlGnBu_09 → 노랑 → 초록 → 파랑
  → 자연 환경, 기후, 생활인구 시각화에 적합

  linear.PuBuGn_09 → 보라 → 파랑 → 청록
  → 다층적 분포, 차분하면서도 다양한 데이터 표현

  linear.BuPu_09 → 하늘색 → 보라
  → 소프트/쿨톤, 중립적 데이터 시각화

  🔹 발산형 (Diverging)

  중앙값(0 같은 기준)에서 양극단 강조 → 긍정/부정, 증가/감소 표현

  linear.RdBu_09 → 빨강 ↔ 파랑
  → 대립적 관계 (긍/부정, 수익/손실)

  linear.RdYlGn_09 → 빨강 → 노랑 → 초록
  → 위험 → 중립 → 안전 단계 표현

  linear.PiYG_09 → 분홍 ↔ 연두
  → 부드러운 대비, 이분법적 분류에 적합

  linear.BrBG_09 → 갈색 ↔ 청록
  → 자연 데이터, 기후/지리학적 대비 시각화

  linear.PuOr_09 → 보라 ↔ 주황
  → 강렬한 대비, 이질적 데이터 표현

  linear.Spectral_09 → 무지개 계열 (파랑 ↔ 초록 ↔ 노랑 ↔ 빨강)
  → 다채로운 데이터, 스펙트럼 분석

## 모델

In [ ]:
# 마비노기_모바일
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import folium
from branca.colormap import linear
from pathlib import Path

# 1. 타겟 정보 구조화 (광고 제품에 따라 이 딕셔너리만 수정하면 됩니다)
# Ensure these keys and values match the actual column names in your data files.
# Based on previous inspections:
# 1. 타겟 정보 구조화
ad_target_info = {
    '인구': [
        '남자20세부터24세생활인구수',
        '남자25세부터29세생활인구수',
        '남자30세부터34세생활인구수',
        '남자35세부터39세생활인구수',
        '여자20세부터24세생활인구수',
        '여자25세부터29세생활인구수'
    ],
    '소득소비': [
        '여가_문화_지출_총금액'
    ],
    '집객시설': [
        '지하철_역_수',
        '버스_정거장_수',
        '대학교_수',
        '극장_수',
        '백화점_수'
    ],
    '점포': [
        '카페',
        '의류',
        '음식점',
        '주점'
    ]
}

# 2. 각 데이터 소스의 최대 점수 정의
max_scores_config = {
    '인구': 40,
    '소득소비': 8,
    '집객시설': 32,
    '점포': 20
}

# 3. 데이터 로딩 함수
def load_data(population_file, income_expenditure_file, facilities_file, stores_file, coordinate_file):
    """
    지정된 CSV 파일들을 Pandas DataFrame으로 로드합니다.
    좌표 파일도 함께 로드합니다.
    """
    df_population = pd.read_csv(population_file)
    df_income_expenditure = pd.read_csv(income_expenditure_file)
    df_facilities = pd.read_csv(facilities_file)
    df_stores = pd.read_csv(stores_file)
    # Load coordinate file with appropriate encoding
    encodings = ["utf-8-sig", "cp949", "utf-8"]
    df_coordinate = None
    last_err = None
    for enc in encodings:
        try:
            df_coordinate = pd.read_csv(coordinate_file, encoding=enc)
            break
        except Exception as e:
            last_err = e
    if df_coordinate is None:
        raise RuntimeError(f"Coordinate CSV 읽기 실패: {coordinate_file} / 마지막 오류: {last_err}")

    return df_population, df_income_expenditure, df_facilities, df_stores, df_coordinate

# 4. 인구 데이터 점수 계산 함수
def calculate_population_scores(df_population, ad_target_info, max_scores_config):
    """
    인구 데이터를 기반으로 월별/행정동별 인구 총점(0-max_scores_config['인구'])을 계산합니다.
    """
    # '연월' 컬럼을 datetime 형식으로 변환
    df_population['연월'] = pd.to_datetime(df_population['연월'], format='%Y%m')

    # 각 행정동별 월별 전체 예측 인구 합계 계산
    # '총_생활인구' 컬럼이 이미 있으므로 이를 사용합니다.
    monthly_population_sum = df_population.groupby(['행정동코드', '행정동명', '연월'])['총생활인구수'].sum().reset_index(name='월별_전체_인구_합계')

    # 각 행정동별 월별 타겟 예측 인구 합계 계산
    target_population_cols = ad_target_info.get('인구', []) # Use .get to handle potential missing key
    # 모든 행정동에 대해 타겟 인구 컬럼이 있는지 확인하고 존재하는 컬럼만 사용
    valid_target_population_cols = [col for col in target_population_cols if col in df_population.columns]

    if not valid_target_population_cols:
        print("Warning: 지정된 타겟 인구 컬럼이 인구 데이터에 없습니다. 타겟 인구 점수는 0으로 계산됩니다.")
        df_population['월별_타겟_인구'] = 0
    else:
        df_population['월별_타겟_인구'] = df_population[valid_target_population_cols].sum(axis=1)

    monthly_target_population_sum = df_population.groupby(['행정동코드', '행정동명', '연월'])['월별_타겟_인구'].sum().reset_index(name='월별_타겟_인구')


    # 전체 예측 인구 합계 DataFrame과 타겟 예측 인구 합계 DataFrame 병합
    merged_monthly_population = pd.merge(monthly_population_sum, monthly_target_population_sum, on=['행정동코드', '행정동명', '연월'])

    # 인구 데이터 점수 범위 정의 (max_scores_config 사용)
    max_score_total = max_scores_config.get('인구', 0) # Use .get
    # 베이스 인구와 타겟 인구의 점수 비중을 5:1로 유지 (25:5)
    # Check if max_score_total is zero to avoid division by zero
    max_score_base_pop = max_score_total * (25/40) if max_score_total > 0 else 0 # Scale 25 based on original total 40
    max_score_target_seg = max_score_total * (15/40) if max_score_total > 0 else 0 # Scale 15 based on original total 40
    min_score_base_pop = 0
    min_score_target_seg = 0


    # 베이스 인구 점수 (0 - max_score_base_pop) 스케일링
    scaler_base_pop_score = MinMaxScaler(feature_range=(min_score_base_pop, max_score_base_pop))
    if merged_monthly_population['월별_전체_인구_합계'].max() == merged_monthly_population['월별_전체_인구_합계'].min():
        print("Warning: '월별_전체_인구_합계' 컬럼에 데이터 변화가 없습니다. 중간값을 할당합니다.")
        merged_monthly_population['베이스_인구_점수'] = (min_score_base_pop + max_score_base_pop) / 2
    else:
        merged_monthly_population['베이스_인구_점수'] = scaler_base_pop_score.fit_transform(merged_monthly_population[['월별_전체_인구_합계']])

    # 타겟 인구 점수 (0 - max_score_target_seg) 스케일링
    scaler_target_seg_score = MinMaxScaler(feature_range=(min_score_target_seg, max_score_target_seg))
    if merged_monthly_population['월별_타겟_인구'].max() == merged_monthly_population['월별_타겟_인구'].min():
         print("Warning: '월별_타겟_인구' 컬럼에 데이터 변화가 없습니다. 중간값을 할당합니다.")
         merged_monthly_population['타겟_인구_점수'] = (min_score_target_seg + max_score_target_seg) / 2
    else:
        merged_monthly_population['타겟_인구_점수'] = scaler_target_seg_score.fit_transform(merged_monthly_population[['월별_타겟_인구']])

    # 인구 총점 (0 - max_score_total) 계산
    merged_monthly_population[f'인구_총점 (0-{max_score_total})'] = merged_monthly_population['베이스_인구_점수'] + merged_monthly_population['타겟_인구_점수']

    # Keep temporary score columns for potential breakdown display if needed
    # merged_monthly_population.drop(columns=['베이스_인구_점수', '타겟_인구_점수'], inplace=True)


    return merged_monthly_population[['행정동코드', '행정동명', '연월', f'인구_총점 (0-{max_score_total})']]

# 5. 소득소비 데이터 점수 계산 함수
def calculate_income_expenditure_scores(df_income_expenditure, ad_target_info, max_scores_config):
    """
    소득소비 데이터를 기반으로 행정동별 소득소비 총점(0-max_scores_config['소득소비'])을 계산합니다.
    """
    # 제품 타겟에 맞춰 가중치를 부여할 소득소비 데이터 컬럼 정의
    income_expenditure_cols_to_score_raw = ad_target_info.get('소득소비', []) # Use .get

    # 데이터프레임에 실제로 존재하는 컬럼만 사용
    income_expenditure_cols_to_score = [col for col in income_expenditure_cols_to_score_raw if col in df_income_expenditure.columns]

    # 소득소비 데이터 점수 범위 정의 (max_scores_config 사용)
    max_score_total = max_scores_config.get('소득소비', 0) # Use .get
    min_score_total = 0

    df_income_expenditure_scores = df_income_expenditure[['행정동_코드', '행정동_코드_명']].copy()

    if not income_expenditure_cols_to_score:
        print("Warning: 소득소비 점수 계산을 위한 관련 컬럼이 데이터에 없습니다. 소득소비 총점은 0으로 계산됩니다.")
        df_income_expenditure_scores[f'소득소비 총점 (0-{max_score_total})'] = 0.0

    else:
        # 각 컬럼의 값을 스케일링 (비율 유지를 위해 각 항목의 최대 점수를 총점 기준으로 분배)
        total_items = len(income_expenditure_cols_to_score)
        max_score_per_item = max_score_total / total_items if total_items > 0 else 0
        min_score_per_item = 0

        score_cols_list = []
        for col in income_expenditure_cols_to_score:
            scaler = MinMaxScaler(feature_range=(min_score_per_item, max_score_per_item))
            score_col_name = f'{col}_점수' # Use a simple temporary name

            if df_income_expenditure[col].max() == df_income_expenditure[col].min():
                print(f"Warning: '{col}' 컬럼에 데이터 변화가 없습니다. 중간값을 할당합니다.")
                df_income_expenditure_scores[score_col_name] = (min_score_per_item + max_score_per_item) / 2
            else:
                df_income_expenditure_scores[score_col_name] = scaler.fit_transform(df_income_expenditure[[col]])
            score_cols_list.append(score_col_name)

        # 스케일링된 소득 및 지출 컬럼의 점수를 합산하여 소득소비 총점 (0-max_score_total) 계산
        df_income_expenditure_scores[f'소득소비 총점 (0-{max_score_total})'] = df_income_expenditure_scores[score_cols_list].sum(axis=1)
        # Drop individual temporary score columns before returning
        df_income_expenditure_scores.drop(columns=score_cols_list, inplace=True)


    df_income_expenditure_scores.rename(columns={'행정동_코드': '행정동코드', '행정동_코드_명': '행정동명'}, inplace=True)

    return df_income_expenditure_scores[['행정동코드', '행정동명', f'소득소비 총점 (0-{max_score_total})']]

# 6. 집객시설 데이터 점수 계산 함수
def calculate_facilities_scores(df_facilities, ad_target_info, max_scores_config):
    """
    집객시설 데이터를 기반으로 행정동별 집객시설 총점(0-max_scores_config['집객시설'])을 계산합니다.
    """
    # 가중치를 부여할 집객시설 데이터 컬럼 정의
    target_facility_cols = ad_target_info.get('집객시설', [])
    # 모든 행정동에 대해 타겟 시설 컬럼이 있는지 확인하고 존재하는 컬럼만 사용
    valid_target_facility_cols = [col for col in target_facility_cols if col in df_facilities.columns]

    # 집객시설 데이터 점수 범위 정의 (max_scores_config 사용)
    min_score_total = 0
    max_score_total = max_scores_config.get('집객시설', 0)

    df_facilities_scores = df_facilities[['행정동_코드', '행정동_코드_명']].copy()

    if max_score_total == 0 or not valid_target_facility_cols:
        print("Warning: 집객시설 점수 비중이 0이거나 지정된 타겟 집객시설 컬럼이 데이터에 없습니다. 집객시설 총점은 0으로 계산됩니다.")
        df_facilities_scores[f'집객시설 총점 (0-{max_score_total})'] = 0.0
    else:
        # 각 컬럼의 값을 스케일링 (비율 유지를 위해 각 항목의 최대 점수를 총점 기준으로 분배)
        total_items = len(valid_target_facility_cols)
        max_score_per_item = max_score_total / total_items if total_items > 0 else 0
        min_score_per_item = 0

        score_cols_list = []
        for col in valid_target_facility_cols:
            scaler = MinMaxScaler(feature_range=(min_score_per_item, max_score_per_item))
            score_col_name = f'{col}_점수' # Use a simple temporary name

            if df_facilities[col].max() == df_facilities[col].min():
                print(f"Warning: '{col}' 컬럼에 데이터 변화가 없습니다. 중간값을 할당합니다.")
                df_facilities_scores[score_col_name] = (min_score_per_item + max_score_per_item) / 2
            else:
                df_facilities_scores[score_col_name] = scaler.fit_transform(df_facilities[[col]])
            score_cols_list.append(score_col_name)

        # 스케일링된 집객시설 점수 합산하여 집객시설 총점 (0-max_score_total) 계산
        df_facilities_scores[f'집객시설 총점 (0-{max_score_total})'] = df_facilities_scores[score_cols_list].sum(axis=1)
        # Drop individual temporary score columns before returning
        df_facilities_scores.drop(columns=score_cols_list, inplace=True)


    df_facilities_scores.rename(columns={'행정동_코드': '행정동코드', '행정동_코드_명': '행정동명'}, inplace=True)

    return df_facilities_scores[['행정동코드', '행정동명', f'집객시설 총점 (0-{max_scores_config["집객시설"]})']]

# 7. 점포 데이터 점수 계산 함수
def calculate_stores_scores(df_stores, ad_target_info, max_scores_config):
    """
    점포 데이터를 기반으로 행정동별 점포 총점(0-max_scores_config['점포'])을 계산합니다.
    """
    # 가중치를 부여할 점포 데이터 컬럼 정의
    target_store_cols = ad_target_info.get('점포', []) # Use .get
    # 모든 행정동에 대해 타겟 점포 컬럼이 있는지 확인하고 존재하는 컬럼만 사용
    valid_target_store_cols = [col for col in target_store_cols if col in df_stores.columns]


    # 점포 데이터 점수 범위 정의 (max_scores_config 사용)
    min_score_total = 0
    max_score_total = max_scores_config.get('점포', 0) # Use .get

    df_stores_scores = df_stores[['행정동_코드', '행정동_코드_명']].copy()

    if max_score_total == 0 or not valid_target_store_cols:
        print("Warning: 점포 점수 비중이 0이거나 지정된 타겟 점포 컬럼이 점포 데이터에 없습니다. 점포 총점은 0으로 계산됩니다.")
        df_stores_scores[f'점포 총점 (0-{max_score_total})'] = 0.0
    else:
        # 각 컬럼의 값을 스케일링 (비율 유지를 위해 각 항목의 최대 점수를 총점 기준으로 분배)
        total_items = len(valid_target_store_cols)
        max_score_per_item = max_score_total / total_items if total_items > 0 else 0
        min_score_per_item = 0

        score_cols_list = []
        for col in valid_target_store_cols:
            scaler = MinMaxScaler(feature_range=(min_score_per_item, max_score_per_item))
            score_col_name = f'{col}_점수' # Use a simple temporary name

            if df_stores[col].max() == df_stores[col].min():
                print(f"Warning: '{col}' 컬럼에 데이터 변화가 없습니다. 중간값을 할당합니다.")
                df_stores_scores[score_col_name] = (min_score_per_item + max_score_per_item) / 2
            else:
                df_stores_scores[score_col_name] = scaler.fit_transform(df_stores[[col]])
            score_cols_list.append(score_col_name)

        # 스케일링된 점포 점수 합산하여 점포 총점 (0-max_score_total) 계산
        df_stores_scores[f'점포 총점 (0-{max_score_total})'] = df_stores_scores[score_cols_list].sum(axis=1)
        # Drop individual temporary score columns before returning
        df_stores_scores.drop(columns=score_cols_list, inplace=True)


    df_stores_scores.rename(columns={'행정동_코드': '행정동코드', '행정동_코드_명': '행정동명'}, inplace=True)

    return df_stores_scores[['행정동코드', '행정동명', f'점포 총점 (0-{max_score_total})']]


# 8. 점수 병합 및 최종 계산 함수
def calculate_final_ad_score(df_population_scores, df_income_expenditure_scores, df_facilities_scores, df_stores_scores, max_scores_config):
    """
    각 데이터 소스의 점수를 병합하고 최종 광고 효율 점수(0-10)를 계산합니다.
    """
    # 월별 인구 점수 평균 계산 (행정동별로 집계)
    population_total_col = f'인구_총점 (0-{max_scores_config.get("인구", 0)})' # Use .get
    df_population_scores_avg = df_population_scores.groupby(['행정동코드', '행정동명'])[population_total_col].mean().reset_index()
    df_population_scores_avg.rename(columns={population_total_col: f'인구_총점 (0-{max_scores_config.get("인구", 0)})_평균'}, inplace=True)


    # 모든 점수 데이터프레임 병합
    df_ad_score = pd.merge(df_population_scores_avg, df_income_expenditure_scores, on=['행정동코드', '행정동명'], how='left')
    df_ad_score = pd.merge(df_ad_score, df_facilities_scores, on=['행정동코드', '행정동명'], how='left')
    df_ad_score = pd.merge(df_ad_score, df_stores_scores, on=['행정동코드', '행정동명'], how='left')

    # 병합 후 발생할 수 있는 NaN 값 처리 (데이터가 없는 행정동은 0으로 처리)
    df_ad_score.fillna(0, inplace=True)

    # 각 카테고리 점수 합산하여 총점 계산
    # Use dynamically generated column names and .get
    total_score_cols = []
    if max_scores_config.get('인구', 0) > 0:
        total_score_cols.append(f'인구_총점 (0-{max_scores_config["인구"]})_평균')
    if max_scores_config.get('소득소비', 0) > 0:
         total_score_cols.append(f'소득소비 총점 (0-{max_scores_config["소득소비"]})')
    if max_scores_config.get('집객시설', 0) > 0:
        total_score_cols.append(f'집객시설 총점 (0-{max_scores_config["집객시설"]})')
    if max_scores_config.get('점포', 0) > 0:
        total_score_cols.append(f'점포 총점 (0-{max_scores_config["점포"]})')

    total_possible_score = sum(max_scores_config.values()) # Sum of all category max scores
    df_ad_score[f'총점 (0-{int(total_possible_score)})'] = df_ad_score[total_score_cols].sum(axis=1)


    # 최종 광고 효율 점수 (0-10점 만점) 스케일링
    min_final_score_range = 0
    max_final_score_range = 10

    # Use the dynamically generated total score column for scaling
    total_score_column_name = f'총점 (0-{int(total_possible_score)})'
    total_score_max = df_ad_score[total_score_column_name].max()
    total_score_min = df_ad_score[total_score_column_name].min()


    scaler_final_score = MinMaxScaler(feature_range=(min_final_score_range, max_final_score_range))

    if total_score_max == total_score_min:
        print("Warning: 총점에 데이터 변화가 없습니다. 중간값 5점을 할당합니다.")
        df_ad_score['광고_효율_점수 (0-10점만점)'] = 5.0
    else:
        df_ad_score['광고_효율_점수 (0-10점만점)'] = scaler_final_score.fit_transform(df_ad_score[[total_score_column_name]])

    return df_ad_score

# 9. 결과 출력 함수
def display_ad_scores(overall_ad_score_data, df_monthly_population_scores, max_scores_config):
    """
    광고 효율 점수 결과를 출력합니다.
    Requires max_scores_config to dynamically find population total column name.
    Includes rank for specific requested district.
    """
    # 전체 행정동 광고 효율 점수 (내림차순 정렬, 전체 기간 평균) with rank
    all_ad_score_sorted = overall_ad_score_data.sort_values(by='광고_효율_점수 (0-10점만점)', ascending=False).reset_index(drop=True).assign(순위=lambda x: x.index + 1)

    # 사용자 입력 받기 (결과 확인용)
    try:
        input_year = int(input("광고 효율 점수를 확인할 연도를 입력하세요 (예: 2025): "))
        input_month = int(input("광고 효율 점수를 확인할 월을 입력하세요 (예: 7): "))
        input_행정동명 = input("광고 효율 점수를 확인할 행정동명을 입력하세요 (예: 역삼1동): ")
        input_date = pd.to_datetime(f'{input_year}-{input_month:02d}-01')

        # 특정 행정동의 월별 인구 점수 출력 (인구 점수 데이터가 있는 경우)
        if df_monthly_population_scores is not None:
             filtered_monthly_population_scores = df_monthly_population_scores[
                (df_monthly_population_scores['연월'] == input_date) &
                (df_monthly_population_scores['행정동명'] == input_행정동명)
            ]
             if not filtered_monthly_population_scores.empty:
                print(f"\n{input_year}년 {input_month}월 {input_행정동명}의 월별 인구 총점:")
                # Dynamically find the population total column using max_scores_config
                pop_total_col_display = f'인구_총점 (0-{max_scores_config.get("인구", 0)})'
                if pop_total_col_display in filtered_monthly_population_scores.columns:
                     display(filtered_monthly_population_scores[['행정동명', '연월', pop_total_col_display]]) # Exclude 행정동코드
                else:
                     print(f"Error: 월별 인구 총점 컬럼 '{pop_total_col_display}'을(를) 찾을 수 없습니다.")
             else:
                print(f"\n{input_year}년 {input_month}월 {input_행정동명}에 대한 월별 인구 총점을 찾을 수 없습니다.")


        # 특정 행정동의 최종 광고 효율 점수 출력 (전체 기간 평균) with rank
        filtered_overall_ad_score = all_ad_score_sorted[
            all_ad_score_sorted['행정동명'] == input_행정동명
        ]
        if not filtered_overall_ad_score.empty:
            print(f"\n{input_행정동명}의 최종 광고 효율 점수 (전체 기간 평균):")
            # Display with Rank and 행정동코드
            display(filtered_overall_ad_score[['순위', '행정동명', '광고_효율_점수 (0-10점만점)']]) # Exclude 행정동코드
        else:
            print(f"\n{input_행정동명}에 대한 전체 광고 효율 점수를 찾을 수 없습니다.")

    except ValueError:
        print("\n잘못된 입력 형식입니다. 연도와 월을 숫자로 입력해주세요.")
    except Exception as e:
        print(f"\n결과 출력 중 오류가 발생했습니다: {e}")


    print("\n전체 행정동 광고 효율 점수 (내림차순 정렬, 전체 기간 평균):")
    # Display the top 50 with rank and 행정동코드
    display(all_ad_score_sorted[['순위', '행정동명', '광고_효율_점수 (0-10점만점)']].head(50)) # Exclude 행정동코드


# 10. 광고 효율 점수 계산 전체 프로세스 실행 함수 (데이터프레임 직접 입력 가능하도록 수정)
def run_ad_score_calculation(population_file, income_expenditure_file, facilities_file, stores_file, coordinate_file, ad_target_info, max_scores_config):
    """
    광고 효율 점수 계산의 전체 프로세스를 실행하고 지도를 생성합니다.
    """
    print("1. 데이터 로딩 중...")
    df_population, df_income_expenditure, df_facilities, df_stores, df_coordinate = load_data(
        population_file, income_expenditure_file, facilities_file, stores_file, coordinate_file
    )
    print("데이터 로딩 완료.")

    print("\n2. 인구 데이터 점수 계산 중...")
    # Pass the original df_population to calculate_population_scores
    df_population_scores_monthly = calculate_population_scores(df_population.copy(), ad_target_info, max_scores_config) # Pass a copy
    print("인구 데이터 점수 계산 완료.")

    print("\n3. 소득소비 데이터 점수 계산 중...")
    df_income_expenditure_scores = calculate_income_expenditure_scores(df_income_expenditure.copy(), ad_target_info, max_scores_config) # Pass a copy
    print("소득소비 데이터 점수 계산 완료.")

    print("\n4. 집객시설 데이터 점수 계산 중...")
    df_facilities_scores = calculate_facilities_scores(df_facilities.copy(), ad_target_info, max_scores_config) # Pass a copy
    print("집객시설 데이터 점수 계산 완료.")

    print("\n5. 점포 데이터 점수 계산 중...")
    df_stores_scores = calculate_stores_scores(df_stores.copy(), ad_target_info, max_scores_config) # Pass a copy
    print("점포 데이터 점수 계산 완료.")

    print("\n6. 점수 병합 및 최종 광고 효율 점수 계산 중...")
    # Pass the monthly population scores to calculate_final_ad_score for averaging
    overall_ad_score_data = calculate_final_ad_score(
        df_population_scores_monthly, df_income_expenditure_scores, df_facilities_scores, df_stores_scores, max_scores_config
    )
    print("점수 병합 및 최종 광고 효율 점수 계산 완료.")

    print("\n7. 좌표 데이터 병합 중...")
    # Merge ad score data with coordinate data
    df_merged = pd.merge(overall_ad_score_data, df_coordinate[['코드', 'X', 'Y']], left_on='행정동코드', right_on='코드', how='left')
    df_merged.drop(columns=['코드'], inplace=True)
    # Handle potential missing coordinates after merge
    df_merged.dropna(subset=['X', 'Y'], inplace=True)
    print("좌표 데이터 병합 완료.")

    print("\n8. 광고 효율 지도 생성 중...")
    # Map visualization code adapted from cell cfH2fdOdVDjz

    # 1) Prepare data for map (using merged df)
    work = df_merged[['행정동명', 'X', 'Y', '광고_효율_점수 (0-10점만점)']].copy()
    work.columns = ['행정동명', '경도', '위도', '점수']  # Standardize column names
    # Ensure numeric types
    work['위도'] = pd.to_numeric(work['위도'], errors='coerce')
    work['경도'] = pd.to_numeric(work['경도'], errors='coerce')
    work['점수'] = pd.to_numeric(work['점수'], errors='coerce')
    work = work.dropna(subset=['위도','경도','점수']).reset_index(drop=True)

    # 2) Map: tiles=None로 시작 후 OSM(한글) + 영문 타일 토글
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles=None)

    # 한글 표기 OpenStreetMap
    folium.TileLayer(
        tiles="OpenStreetMap",
        name="OSM (한글표기)"
    ).add_to(m)

    # 참고용 영문 타일(토글)
    folium.TileLayer(
        tiles="CartoDB positron",
        name="영문지도"
    ).add_to(m)

    # 3) 점수→반지름/색상 매핑
    s = work['점수'].astype(float)
    smin, smax = float(np.nanmin(s)), float(np.nanmax(s))
    R_MIN, R_MAX = 6, 28
    def scale_r(v, vmin=smin, vmax=smax, rmin=R_MIN, rmax=R_MAX):
        if not np.isfinite(v): return rmin
        if vmax <= vmin: return (rmin + rmax)/2
        return rmin + (v - vmin) * (rmax - rmin) / (vmax - vmin)


    cmap = linear.Blues_09.scale(smin, smax)


    for _, row in work.iterrows():
        r = scale_r(row['점수'])
        color = cmap(row['점수'])
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=r,
            color=color, fill=True, fill_color=color, fill_opacity=0.65, weight=1,
            tooltip=f"{row['행정동명']} : {row['점수']:.2f}",
            popup=folium.Popup(f"<b>{row['행정동명']}</b><br>점수: {row['점수']:.2f}", max_width=260)
        ).add_to(m)

    cmap.caption = "광고 효율 점수"
    cmap.add_to(m)
    folium.LayerControl(position="topleft", collapsed=False).add_to(m)

    # Save the map to an HTML file
    OUT_HTML = Path("seoul_ad_efficiency_map.html")
    m.save(str(OUT_HTML))
    print(f"✅ 지도 저장: {OUT_HTML.resolve()}")
    print("광고 효율 지도 생성 완료.")

    print("\n9. 결과 출력:")
    # Pass the monthly population scores to display_ad_scores
    display_ad_scores(overall_ad_score_data, df_population_scores_monthly, max_scores_config)

    # Display the map in the notebook output
    # display(m) # Commented out to prevent automatic display


    return overall_ad_score_data, df_population, df_income_expenditure, df_facilities, df_stores, df_coordinate, m # Return all dataframes including the map object


# 파일 경로 정의
population_file = '생활인구_통합본.csv'
income_expenditure_file = '소득소비_전처리.csv'
facilities_file = '집객시설_전처리.csv'
stores_file = '점포_전처리.csv'
coordinate_file = '서울시_행정동_중심점_2017.csv' # Add coordinate file

In [ ]:
# 광고 효율 점수 계산 전체 프로세스 실행
final_ad_scores_df, df_population, df_income_expenditure, df_facilities, df_stores, df_coordinate, ad_efficiency_map = run_ad_score_calculation(
    population_file, income_expenditure_file, facilities_file, stores_file, coordinate_file, ad_target_info, max_scores_config
)

1. 데이터 로딩 중...
데이터 로딩 완료.

2. 인구 데이터 점수 계산 중...
인구 데이터 점수 계산 완료.

3. 소득소비 데이터 점수 계산 중...
소득소비 데이터 점수 계산 완료.

4. 집객시설 데이터 점수 계산 중...
집객시설 데이터 점수 계산 완료.

5. 점포 데이터 점수 계산 중...
점포 데이터 점수 계산 완료.

6. 점수 병합 및 최종 광고 효율 점수 계산 중...
점수 병합 및 최종 광고 효율 점수 계산 완료.

7. 좌표 데이터 병합 중...
좌표 데이터 병합 완료.

8. 광고 효율 지도 생성 중...
✅ 지도 저장: /content/seoul_ad_efficiency_map.html
광고 효율 지도 생성 완료.

9. 결과 출력:
광고 효율 점수를 확인할 연도를 입력하세요 (예: 2025): 2025
광고 효율 점수를 확인할 월을 입력하세요 (예: 7): 8
광고 효율 점수를 확인할 행정동명을 입력하세요 (예: 역삼1동): 사직동

2025년 8월 사직동의 월별 인구 총점:


,행정동명,연월,인구_총점 (0-40)
67,사직동,2025-08-01,6.835274



사직동의 최종 광고 효율 점수 (전체 기간 평균):


,순위,행정동명,광고_효율_점수 (0-10점만점)
40,41,사직동,2.858509



전체 행정동 광고 효율 점수 (내림차순 정렬, 전체 기간 평균):


,순위,행정동명,광고_효율_점수 (0-10점만점)
0,1,서교동,10.000000
1,2,역삼1동,9.770349
2,3,여의동,8.353825
3,4,신촌동,7.650769
4,5,가산동,6.228595
5,6,한강로동,5.480110
6,7,소공동,5.344628
7,8,종로1.2.3.4가동,5.236374
8,9,서초3동,4.963250
9,10,가양1동,4.851462


#### 지도 시각화

In [ ]:
# 생성된 광고 효율 지도를 출력
display(ad_efficiency_map)